In [1]:
import os, json, time, traceback, argparse
# import openai
from tqdm import tqdm
from glob import glob
import torch
import numpy as np
import pandas as pd
import re
# from modelscope import AutoModelForCausalLM
# from modelscope import AutoTokenizer
# from modelscope import GenerationConfig
# from modelscope import snapshot_download
import transformers
from multiprocessing import Process, Pool
import multiprocessing
from collections import Counter

In [2]:
# glm_device = "cuda:0"
# glm_tokenizer = transformers.AutoTokenizer.from_pretrained("GPTs/chatglm3-6b", trust_remote_code=True)
# glm_model = transformers.AutoModel.from_pretrained("GPTs/chatglm3-6b", device_map="cuda:0", trust_remote_code=True).half()
# glm_model = glm_model.eval()
# # response, history = glm_model.chat(glm_tokenizer, "你好", history=[])
# def chat_glm3(glm_model, glm_tokenizer, glm_device, prompt, history=[]):
#     response, history = glm_model.chat(glm_tokenizer, prompt, history=history)
#     # print(history)
#     return response, history
# model_a_name = "chatglm3-6b"

In [3]:
# yi_device = "cuda:0"
# yi_tokenizer = transformers.AutoTokenizer.from_pretrained('GPTs/Yi-6B', use_fast=False)
# yi_model = transformers.AutoModelForCausalLM.from_pretrained('GPTs/Yi-6B', device_map=yi_device, torch_dtype='auto').eval()
# def chat_yi(yi_model, yi_tokenizer, yi_device, prompt, messages=None):
#     if messages == None:
#         messages = [{"role": "user", "content": prompt}]
#     else:
#         messages.append({"role": "user", "content": prompt})
#     input_ids = yi_tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
#     output_ids = yi_model.generate(input_ids.to('cuda'))
#     response = yi_tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
#     messages.append({"role": "assistant", "content": response})
#     return response, messages
# model_a_name = "yi-6b"

In [4]:
# gemma_device = "cuda:0"
# gemma_tokenizer = transformers.AutoTokenizer.from_pretrained('GPTs/gemma-7b')
# gemma_model = transformers.AutoModelForCausalLM.from_pretrained('GPTs/gemma-7b', device_map=gemma_device, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2").eval()
# def chat_gemma(gemma_model, gemma_tokenizer, gemma_device, prompt, messages=None):
#     if messages == None:
#         messages = [{"role": "user", "content": prompt}]
#     else:
#         messages.append({"role": "user", "content": prompt})
#     outputs = gemma_model.generate(input_ids=gemma_tokenizer.encode(
#         gemma_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True), 
#         add_special_tokens=False, return_tensors="pt").to(gemma_model.device), max_new_tokens=150)
#     response = gemma_tokenizer.decode(outputs[0])
#     messages.append({"role": "assistant", "content": response})
#     return response, messages
# model_a_name = "gemma-7b"

In [5]:
# phi_device = "cuda:0"
# phi_model = transformers.AutoModelForCausalLM.from_pretrained("GPTs/phi-1", torch_dtype="auto", trust_remote_code=True, device_map=phi_device)
# phi_tokenizer = transformers.AutoTokenizer.from_pretrained("GPTs/phi-1", trust_remote_code=True)
# model_a_name = "phi-1b"

In [6]:
# qw_device = "cuda:0"
# qw_model = transformers.AutoModelForCausalLM.from_pretrained("GPTs/qwen1.5-7b", torch_dtype="auto", device_map=qw_device, attn_implementation="flash_attention_2")
# qw_tokenizer = transformers.AutoTokenizer.from_pretrained("GPTs/qwen1.5-7b")
# # Specify hyperparameters for generation. But if you use transformers>=4.32.0, there is no need to do this.
# # model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-14B-Chat", trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参
# """
# def chat_qwen(qw_model, qw_tokenizer, qw_device, prompt, messages=None):
#     if messages is None:
#         messages = [{"role": "system", "content": "You are a helpful assistant."}]
#     messages.append({"role": "user", "content": prompt})
#     new_input = qw_tokenizer([qw_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)], return_tensors="pt").to(qw_device)
#     resp = qw_tokenizer.batch_decode([output_ids[len(input_ids):] for input_ids, output_ids in zip(new_input.input_ids, qw_model.generate(new_input.input_ids, max_new_tokens=512, pad_token_id=qw_tokenizer.eos_token_id))], 
#                                  skip_special_tokens=True)[0]
#     messages.append({"role": "assistant", "content": resp})
#     return resp, messages
# """
# model_a_name = "qwen1.5-7b"

In [7]:
# tiny_device = "cuda:0"
# tiny_model = transformers.AutoModelForCausalLM.from_pretrained('GPTs/TinyLlama-1.1B', device_map=tiny_device, torch_dtype='auto').eval()
# tiny_tokenizer = transformers.AutoTokenizer.from_pretrained('GPTs/TinyLlama-1.1B', truncation_side='left')
# model_b_name = 'TinyLlama-1.1B'

In [8]:
# llama_device = "cuda:1"
# llama_model = transformers.AutoModelForCausalLM.from_pretrained('GPTs/llama2-7b', device_map=llama_device, torch_dtype='auto', attn_implementation="flash_attention_2").eval()
# llama_tokenizer = transformers.AutoTokenizer.from_pretrained('GPTs/llama2-7b')#, truncation_side='left')
# model_b_name = 'llama2-7b'

In [9]:
baichuan_device = "cuda:0"
baichuan_tokenizer = transformers.AutoTokenizer.from_pretrained("GPTs/Baichuan2-7B", trust_remote_code=True)
baichuan_model = transformers.AutoModelForCausalLM.from_pretrained("GPTs/Baichuan2-7B", device_map=baichuan_device, torch_dtype=torch.bfloat16, trust_remote_code=True).half()
baichuan_model.generation_config = transformers.GenerationConfig.from_pretrained("GPTs/Baichuan2-7B")
baichuan_model = baichuan_model.eval()
model_a_name = "Baichuan2-7B"

In [10]:
mistral_device = "cuda:1"
mistral_model = transformers.AutoModelForCausalLM.from_pretrained("GPTs/Mistral-v0.2-7B", torch_dtype="auto", device_map=mistral_device, attn_implementation="flash_attention_2")
mistral_tokenizer = transformers.AutoTokenizer.from_pretrained("GPTs/Mistral-v0.2-7B")
model_b_name = "Mistral-v0.2-7B"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
max_round = 8
timestamp = 20240329
result_dir = "Results/GameLLM"
save_last_n_round = 1
if not os.path.exists(result_dir): os.mkdir(result_dir)
"""original Debate logic: choose most frequent as game ans when no consensus"""

'original Debate logic: choose most frequent as game ans when no consensus'

## MMLU

In [12]:
#"""in the form (X) <-> in between the brakets """
"""put your answer ++choice++
A) -> (A)
in the form (A) or (B) or (C) or (D)
"""
def parse_mmlu(df, ix, subject_name):
    question = df.iloc[ix, 0]
    a = df.iloc[ix, 1]
    b = df.iloc[ix, 2]
    c = df.iloc[ix, 3]
    d = df.iloc[ix, 4]
    question = "You are an expert in {}. Can you answer the following question as accurately as possible? {}: (A) {}, (B) {}, (C) {}, (D) {} Explain your answer, putting the answer choice in the form (A) or (B) or (C) or (D) at the end of your response.".format(subject_name, question, a, b, c, d)
    answer = df.iloc[ix, 5]
    return question, answer

def game_prompt_mmlu(another_resp):
    # if len(agents) == 0:
    #     return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}
    prefix_string = "These are the solutions to the problem from other agents: \n\n One agent solution: ```{}```".format(another_resp)
    prefix_string += """\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. Put your answer choice in the form (A) or (B) or (C) or (D) at the end of your response."""
    return prefix_string

def parse_choice_ans(input_str):
    pattern = r'\(\s{0,}(\w)\s{0,}\)'
    matches = re.findall(pattern, input_str)
    solution = None
    for match_str in matches[::-1]:
        solution = match_str.upper()
        if solution: break
    return solution

In [15]:
def main_run(model_a, model_b, tkz_a, tkz_b, device_a, device_b, chat_model_a, chat_model_b):
    # task paths
    task_paths = glob("Corpus/MMLU/test/*.csv")
    subjects = list(map(lambda x: x.split('/')[-1].split('_test.csv')[0], task_paths))
    # build result df
    result_csv_filename = f"{model_a_name}_{model_b_name}_{timestamp}_{max_round}_Last{save_last_n_round}Round.csv"
    csv_path = os.path.join(result_dir, result_csv_filename)
    if os.path.exists(csv_path):
        result_df = pd.read_csv(csv_path)
    else:
        result_df = pd.DataFrame(columns=['subject','ith','label','a_first_ans','b_first_ans','a_game_ans','b_game_ans','rounds'])
        result_df.to_csv(csv_path, index=False)
    def write_to_result_df(result_df, subject, i, answer, a_first_ans, b_first_ans, a_game_answer, b_game_answer, g):
        result_df = pd.concat([result_df, pd.DataFrame([{
                'subject': subject, 'ith':i, 'label':answer,
                'a_first_ans':a_first_ans,'b_first_ans':b_first_ans,
                'a_game_ans':a_game_answer,'b_game_ans':b_game_answer,'rounds':g
            }])],ignore_index=True)
        result_df.to_csv(os.path.join(result_dir, result_csv_filename), index=False)
        return result_df
    
    for task_path, subject in zip(task_paths, subjects):
        # fetch statistic
        df = pd.read_csv(task_path, header=None)
        if (result_lines:=len(result_df[result_df['subject']==subject]))>0:
            if result_lines==len(df): continue
            label_so_far = result_df[result_df['subject']==subject].label.to_list()
            model_a_so_far = result_df[result_df['subject']==subject].a_first_ans.to_list()
            model_b_so_far = result_df[result_df['subject']==subject].b_first_ans.to_list()
            game_a_so_far = result_df[result_df['subject']==subject].a_game_ans.to_list()
            game_b_so_far = result_df[result_df['subject']==subject].b_game_ans.to_list()
            a_correct = list(map(lambda x: x[0]==x[1], zip(label_so_far, model_a_so_far))).count(True)
            b_correct = list(map(lambda x: x[0]==x[1], zip(label_so_far, model_b_so_far))).count(True)
            game_correct = list(map(lambda x: x[0]==x[1]==x[2], zip(label_so_far, game_a_so_far, game_b_so_far))).count(True)
            # subject_start_idx = len(label_so_far)
            # if len(df) == subject_start_idx:
            #     continue
        else:
            a_correct, b_correct, game_correct, subject_start_idx = 0,0,0,0
        # print(f"Current subjuect: {subject}")
        # into one subject
        subj_slice = result_df[result_df['subject']==subject]
        for idx in (pbar:=tqdm(range(len(df)))):
            if len(subj_slice)>0:    # skip processed line by line
                if idx in subj_slice['ith'].values: continue
            # save dialog
            dialog_dir = os.path.join(result_dir, f"dialog_{timestamp}_LastOnly")
            if not os.path.exists(dialog_dir): os.mkdir(dialog_dir)
            dialog_filename = f"{model_a_name}_{model_b_name}_{timestamp}_{subject}_{idx}_LastOnly.txt"
            dialog_fp = open(os.path.join(dialog_dir, dialog_filename), 'w+', encoding="utf-8")
            # game rounds
            answers_for_majority = []
            question, answer = parse_mmlu(df, idx, subject.replace("_", " "))
            for round in range(max_round):
                if round == 0:
                    pool = Pool(processes=2)
                    result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, question,))
                    result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, question,))
                    pool.close()
                    pool.join()
                    a_resp, a_history = result_a.get()
                    b_resp, b_msgs = result_b.get()
                    # a_resp, a_history = chat_model_a(model_a, tkz_a, device_a, question)
                    # b_resp, b_msgs = chat_model_b(model_b, tkz_b, device_b, question)
                else:    # ans_a != ans_b
                    if round > save_last_n_round:    # shave history
                        a_history = a_history[:(3 if a_history[0]['role'] == "system" else 2)] + a_history[-2*save_last_n_round:]
                        b_msgs = b_msgs[:(3 if b_msgs[0]['role'] == "system" else 2)] + b_msgs[-2*save_last_n_round:]
                    game_a_prompt, game_b_prompt = game_prompt_mmlu(b_resp), game_prompt_mmlu(a_resp)
                    # a_resp, a_history = chat_model_a(model_a, tkz_a, device_a, game_a_prompt, a_history)
                    # b_resp, b_msgs = chat_model_b(model_b, tkz_b, device_b, game_b_prompt, b_msgs)
                    pool = Pool(processes=2)
                    result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, game_a_prompt, a_history,))
                    result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, game_b_prompt, b_msgs,))
                    pool.close()
                    pool.join()
                    a_resp, a_history = result_a.get()
                    b_resp, b_msgs = result_b.get()
                # save response
                dialog_fp.write(f"a: {a_resp}\nb: {b_resp}\n\n")
                ans_a, ans_b = parse_choice_ans(a_resp), parse_choice_ans(b_resp)
                # save answer for voting
                if 'MMLU' in task_path.upper():
                    if ans_a in ['A', 'B', 'C', 'D']: answers_for_majority.append(ans_a)
                    if ans_b in ['A', 'B', 'C', 'D']: answers_for_majority.append(ans_b)
                # control game flow
                if round == 0: 
                    a_first_ans, b_first_ans = ans_a, ans_b
                    if ans_a == answer: a_correct += 1
                    if ans_b == answer: b_correct += 1
                if ans_a == ans_b:
                    if ans_a == answer:
                        game_correct += 1
                    result_df = write_to_result_df(result_df, subject, idx, answer, a_first_ans, b_first_ans, ans_a, ans_b, round)
                    break
                # majority vote when no converge
                if round == max_round -1:
                    try:
                        v = Counter(answers_for_majority)
                        vote_result = v.most_common(1)[0][0]
                    except:
                        vote_result = np.random.choice(["A","B","C","D"])
                    result_df = write_to_result_df(result_df, subject, idx, answer, a_first_ans, b_first_ans, vote_result, vote_result, round)
                    answers_for_majority.clear()
                    break
            # Destructor
            del a_history, b_msgs
            dialog_fp.close()
            pbar.set_description(f"Subject: {subject} Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")
        print(f"Subject: {subject} Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")

In [ ]:
from ChatLLM import chat_baichuan, chat_qwen, chat_phi, chat_llama
if __name__ == "__main__":
    if multiprocessing.get_start_method() != 'spawn':
        multiprocessing.set_start_method('spawn', force=True)
    while True:
        try:
            main_run(
                model_a=qw_model, 
                model_b=llama_model, 
                tkz_a=qw_tokenizer, 
                tkz_b=llama_tokenizer, 
                device_a=qw_device, 
                device_b=llama_device, 
                chat_model_a=chat_qwen, 
                chat_model_b=chat_llama
            )
            break
        except torch.cuda.OutOfMemoryError:
            pass
        except ValueError:
            pass

Subject: professional_law Acc: Game=0.379, qwen1.5-7b=0.361, llama2-7b=0.321:  33%|███▎      | 499/1534 [15:47<32:44,  1.90s/it]   
 33%|███▎      | 499/1534 [00:54<01:53,  9.11it/s]
Subject: professional_law Acc: Game=0.383, qwen1.5-7b=0.361, llama2-7b=0.319:  33%|███▎      | 501/1534 [02:12<04:33,  3.78it/s]
 33%|███▎      | 501/1534 [00:58<02:00,  8.60it/s]
Subject: professional_law Acc: Game=0.378, qwen1.5-7b=0.361, llama2-7b=0.316:  33%|███▎      | 510/1534 [04:20<08:43,  1.96it/s]
 33%|███▎      | 510/1534 [00:50<01:40, 10.18it/s]
Subject: professional_law Acc: Game=0.380, qwen1.5-7b=0.360, llama2-7b=0.315:  33%|███▎      | 511/1534 [01:19<02:38,  6.46it/s]
 33%|███▎      | 511/1534 [00:50<01:41, 10.12it/s]
Subject: professional_law Acc: Game=0.383, qwen1.5-7b=0.358, llama2-7b=0.317:  34%|███▎      | 514/1534 [02:35<05:07,  3.31it/s]
Subject: professional_law Acc: Game=0.383, qwen1.5-7b=0.358, llama2-7b=0.317:  34%|███▎      | 517/1534 [02:43<05:20,  3.17it/s]
Subject: profession

Subject: professional_law Acc: Game=0.400, qwen1.5-7b=0.379, llama2-7b=0.337


Subject: college_medicine Acc: Game=0.500, qwen1.5-7b=0.500, llama2-7b=0.000:   1%|          | 2/173 [01:41<2:24:09, 50.58s/it]
Subject: college_medicine Acc: Game=0.500, qwen1.5-7b=0.667, llama2-7b=0.389:  10%|█         | 18/173 [06:54<59:27, 23.02s/it]  
 10%|█         | 18/173 [00:59<08:32,  3.31s/it]
Subject: college_medicine Acc: Game=0.474, qwen1.5-7b=0.632, llama2-7b=0.368:  11%|█         | 19/173 [01:03<08:34,  3.34s/it]
Subject: college_medicine Acc: Game=0.450, qwen1.5-7b=0.600, llama2-7b=0.350:  12%|█▏        | 20/173 [01:18<09:59,  3.92s/it]
Subject: college_medicine Acc: Game=0.500, qwen1.5-7b=0.625, llama2-7b=0.375:  14%|█▍        | 24/173 [02:16<14:09,  5.70s/it]
Subject: college_medicine Acc: Game=0.520, qwen1.5-7b=0.640, llama2-7b=0.400:  14%|█▍        | 25/173 [01:11<07:00,  2.84s/it]
Subject: college_medicine Acc: Game=0.552, qwen1.5-7b=0.655, llama2-7b=0.379:  17%|█▋        | 29/173 [03:30<17:23,  7.25s/it]
Subject: college_medicine Acc: Game=0.562, qwen1.5-7b=0.625

Subject: college_medicine Acc: Game=0.584, qwen1.5-7b=0.520, llama2-7b=0.428


  0%|          | 0/245 [00:52<?, ?it/s]
Subject: security_studies Acc: Game=1.000, qwen1.5-7b=1.000, llama2-7b=0.000:   0%|          | 1/245 [01:27<5:57:39, 87.95s/it]
Subject: security_studies Acc: Game=1.000, qwen1.5-7b=1.000, llama2-7b=0.500:   1%|          | 2/245 [01:15<2:32:44, 37.71s/it]
Subject: security_studies Acc: Game=0.750, qwen1.5-7b=0.750, llama2-7b=0.250:   2%|▏         | 4/245 [01:29<1:30:20, 22.49s/it]
  2%|▏         | 4/245 [00:47<47:32, 11.83s/it]
Subject: security_studies Acc: Game=0.750, qwen1.5-7b=0.750, llama2-7b=0.375:   3%|▎         | 8/245 [02:33<1:15:52, 19.21s/it]
Subject: security_studies Acc: Game=0.700, qwen1.5-7b=0.600, llama2-7b=0.400:   4%|▍         | 10/245 [01:59<46:43, 11.93s/it]
  4%|▍         | 10/245 [00:51<20:07,  5.14s/it]
Subject: security_studies Acc: Game=0.667, qwen1.5-7b=0.533, llama2-7b=0.400:   6%|▌         | 15/245 [03:30<53:45, 14.02s/it]
Subject: security_studies Acc: Game=0.700, qwen1.5-7b=0.600, llama2-7b=0.450:   8%|▊         | 20

...

Subject: medical_genetics Acc: Game=1.000, qwen1.5-7b=0.778, llama2-7b=0.889:   9%|▉         | 9/100 [03:09<32:01, 21.11s/it]
Subject: medical_genetics Acc: Game=0.833, qwen1.5-7b=0.583, llama2-7b=0.750:  12%|█▏        | 12/100 [01:42<12:35,  8.58s/it]
Subject: medical_genetics Acc: Game=0.765, qwen1.5-7b=0.471, llama2-7b=0.824:  17%|█▋        | 17/100 [02:39<13:00,  9.41s/it]
Subject: medical_genetics Acc: Game=0.792, qwen1.5-7b=0.542, llama2-7b=0.792:  24%|██▍       | 24/100 [03:07<09:55,  7.83s/it]
Subject: medical_genetics Acc: Game=0.808, qwen1.5-7b=0.538, llama2-7b=0.808:  26%|██▌       | 26/100 [01:29<04:14,  3.44s/it]
Subject: medical_genetics Acc: Game=0.727, qwen1.5-7b=0.515, llama2-7b=0.667:  33%|███▎      | 33/100 [03:24<06:55,  6.20s/it]
Subject: medical_genetics Acc: Game=0.750, qwen1.5-7b=0.528, llama2-7b=0.667:  36%|███▌      | 36/100 [01:37<02:53,  2.70s/it]
Subject: medical_genetics Acc: Game=0.730, qwen1.5-7b=0.514, llama2-7b=0.649:  37%|███▋      | 37/100 [01:14<02:

Subject: medical_genetics Acc: Game=0.660, qwen1.5-7b=0.550, llama2-7b=0.500


  0%|          | 0/100 [01:00<?, ?it/s]
Subject: college_chemistry Acc: Game=0.500, qwen1.5-7b=0.500, llama2-7b=0.250:   8%|▊         | 8/100 [03:18<37:57, 24.76s/it]
  8%|▊         | 8/100 [00:49<09:31,  6.21s/it]
Subject: college_chemistry Acc: Game=0.400, qwen1.5-7b=0.500, llama2-7b=0.200:  10%|█         | 10/100 [01:34<14:09,  9.43s/it]
Subject: college_chemistry Acc: Game=0.364, qwen1.5-7b=0.455, llama2-7b=0.182:  11%|█         | 11/100 [01:29<12:00,  8.10s/it]
 11%|█         | 11/100 [00:47<06:25,  4.33s/it]
Subject: college_chemistry Acc: Game=0.462, qwen1.5-7b=0.462, llama2-7b=0.308:  13%|█▎        | 13/100 [01:38<10:59,  7.58s/it]
Subject: college_chemistry Acc: Game=0.500, qwen1.5-7b=0.500, llama2-7b=0.389:  18%|█▊        | 18/100 [02:32<11:35,  8.48s/it]
 18%|█▊        | 18/100 [00:48<03:39,  2.68s/it]
Subject: college_chemistry Acc: Game=0.474, qwen1.5-7b=0.474, llama2-7b=0.368:  19%|█▉        | 19/100 [00:54<03:51,  2.86s/it]
Subject: college_chemistry Acc: Game=0.500, qwe

Subject: college_chemistry Acc: Game=0.410, qwen1.5-7b=0.370, llama2-7b=0.300


Subject: moral_scenarios Acc: Game=1.000, qwen1.5-7b=1.000, llama2-7b=0.000:   0%|          | 1/895 [01:17<19:11:20, 77.27s/it]
  0%|          | 1/895 [00:55<13:53:28, 55.94s/it]
Subject: moral_scenarios Acc: Game=0.667, qwen1.5-7b=0.500, llama2-7b=0.333:   1%|          | 6/895 [02:57<7:18:49, 29.62s/it]
Subject: moral_scenarios Acc: Game=0.556, qwen1.5-7b=0.444, llama2-7b=0.222:   1%|          | 9/895 [02:39<4:22:29, 17.78s/it]
  1%|          | 9/895 [00:45<1:15:06,  5.09s/it]
Subject: moral_scenarios Acc: Game=0.538, qwen1.5-7b=0.462, llama2-7b=0.231:   1%|▏         | 13/895 [02:01<2:17:49,  9.38s/it]
  1%|▏         | 13/895 [00:43<49:33,  3.37s/it]
Subject: moral_scenarios Acc: Game=0.571, qwen1.5-7b=0.500, llama2-7b=0.286:   2%|▏         | 14/895 [00:51<53:59,  3.68s/it]
  2%|▏         | 14/895 [00:44<46:43,  3.18s/it]
Subject: moral_scenarios Acc: Game=0.611, qwen1.5-7b=0.444, llama2-7b=0.278:   2%|▏         | 18/895 [02:32<2:03:56,  8.48s/it]
  2%|▏         | 18/895 [00:48<39:20,

## GSM8k

In [26]:
save_last_n_round = 1

from fastparquet import ParquetFile
def parse_gsm(df, ix):
    question = df.iloc[ix, 0]
    answer = df.iloc[ix, 1].split("####")[-1].strip()
    question = f"You are an expert in math. Please solve the following math problem step by step. {question}\nExplain your reasoning. "
    question += "Your final answer should be a single numerical number in the form of {answer} at the end of your response."
    question += "Make sure that the last numerical number in your response is your final answer."
    return question, answer

def game_prompt_gsm(another_resp):
    prefix_string = "These are the solutions to the problem from other agents: \n\n One agent solution: ```{}```".format(another_resp)
    prefix_string += "\n\n Using the solutions from other agents as additional information, can you provide your answer to the math problem? "
    prefix_string += "Examine your solution and other agents' solution step by step. "
    prefix_string += "Your final answer should be a single numerical number in the form of {answer} at the end of your response. "
    prefix_string += "Make sure that the last numerical number in your response is your final answer."
    return prefix_string

def parse_gsm_ans(s):
    s = s.replace(",", "").replace(r"\$", "")    # case: 70,000 $18
    pattern1 = r"\{[+-]?\d+\.?\d*\}"    # fix minus number
    pattern2 = r"[+-]?\d+\.?\d*"
    if (matches := re.findall(pattern1, s)):
        return matches[-1].replace("{", "").replace("}", "")
    elif(matches := re.findall(pattern2, s)):
        return matches[-1].replace("{", "").replace("}", "")
    return None

def gsm_ans_is_equal(resp, label):
    if resp is not None and label is not None:
        if type(label) == str: label = label.replace(",", "")
        if type(resp) == str: resp = resp.replace(",", "")
        return np.abs(float(label)-float(resp))<1e-5
    return False

# for ix in range(len(gsm_test_df)):
#     q, a = parse_gsm(gsm_test_df,ix)
#     if a[0] == '-': print(a)
#     # if "####" in gsm_test_df.iloc[ix, 1]: print(ix)

def main_gsm(model_a, model_b, tkz_a, tkz_b, device_a, device_b, chat_model_a, chat_model_b):
    gsm_test_df = ParquetFile(os.path.expanduser('~/Corpus/gsm8k/main/test-00000-of-00001.parquet')).to_pandas()
    # build result df
    result_csv_filename = f"GSM_{model_a_name}_{model_b_name}_{timestamp}_{max_round}_Last{save_last_n_round}Round.csv"
    csv_path = os.path.join(result_dir, result_csv_filename)
    if os.path.exists(csv_path):
        result_df = pd.read_csv(csv_path)
    else:
        result_df = pd.DataFrame(columns=['ith','label','a_first_ans','b_first_ans','a_game_ans','b_game_ans','rounds'])
        result_df.to_csv(csv_path, index=False)
    def gsm_to_result_df(result_df, i, answer, a_first_ans, b_first_ans, a_game_answer, b_game_answer, g):
        result_df = pd.concat([result_df, pd.DataFrame([{
                'ith':i, 'label':answer,
                'a_first_ans':a_first_ans,'b_first_ans':b_first_ans,
                'a_game_ans':a_game_answer,'b_game_ans':b_game_answer,'rounds':g
            }])],ignore_index=True)
        result_df.to_csv(os.path.join(result_dir, result_csv_filename), index=False)
        return result_df

    restart = True
    for idx in (pbar:=tqdm(range(len(gsm_test_df)))):
        if len(result_df)>0:    # skip processed line by line
            if idx in result_df['ith'].values: continue
            if restart:
                label_so_far = result_df.label.to_list()
                model_a_so_far = result_df.a_first_ans.to_list()
                model_b_so_far = result_df.b_first_ans.to_list()
                game_a_so_far = result_df.a_game_ans.to_list()
                game_b_so_far = result_df.b_game_ans.to_list()
                a_correct = list(map(lambda x: gsm_ans_is_equal(x[0],x[1]), zip(label_so_far, model_a_so_far))).count(True)
                b_correct = list(map(lambda x: gsm_ans_is_equal(x[0],x[1]), zip(label_so_far, model_b_so_far))).count(True)
                game_correct = list(map(lambda x: gsm_ans_is_equal(x[0],x[1]) and gsm_ans_is_equal(x[2],x[1]), 
                                        zip(label_so_far, game_a_so_far, game_b_so_far))).count(True)
                restart = False
        else:
            a_correct, b_correct, game_correct, subject_start_idx = 0,0,0,0
        # save dialog
        dialog_dir = os.path.join(result_dir, f"dialog_{timestamp}_LastOnly")
        if not os.path.exists(dialog_dir): os.mkdir(dialog_dir)
        dialog_filename = f"GSM_{model_a_name}_{model_b_name}_{timestamp}_{idx}_LastOnly.txt"
        dialog_fp = open(os.path.join(dialog_dir, dialog_filename), 'w+', encoding="utf-8")
        # game rounds
        answers_for_majority = []
        question, answer = parse_gsm(gsm_test_df,idx)
        for round in range(max_round):
            if round == 0:
                pool = Pool(processes=2)
                result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, question,))
                result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, question,))
                pool.close()
                pool.join()
                a_resp, a_history = result_a.get()
                b_resp, b_msgs = result_b.get()
            else:    # ans_a != ans_b
                if round > save_last_n_round:    # shave history
                    # print(b_msgs)
                    # return
                    a_history = a_history[:(3 if a_history[0]['role'] == "system" else 2)] + a_history[-2*save_last_n_round:]
                    b_msgs = b_msgs[:(3 if b_msgs[0]['role'] == "system" else 2)] + b_msgs[-2*save_last_n_round:]
                game_a_prompt, game_b_prompt = game_prompt_gsm(b_resp), game_prompt_gsm(a_resp)
                pool = Pool(processes=2)
                result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, game_a_prompt, a_history))
                result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, game_b_prompt, b_msgs))
                pool.close()
                pool.join()
                a_resp, a_history = result_a.get()
                b_resp, b_msgs = result_b.get()
            # save response
            dialog_fp.write(f"a: {a_resp}\nb: {b_resp}\n\n")
            ans_a, ans_b = parse_gsm_ans(a_resp), parse_gsm_ans(b_resp)
            # save answer for voting
            answers_for_majority.append(ans_a)
            answers_for_majority.append(ans_b)
            # control game flow
            if round == 0: 
                a_first_ans, b_first_ans = ans_a, ans_b
                if gsm_ans_is_equal(ans_a, answer.replace(',',"")): a_correct += 1
                if gsm_ans_is_equal(ans_b, answer.replace(',',"")): b_correct += 1
            if gsm_ans_is_equal(ans_a, ans_b):
                if gsm_ans_is_equal(ans_a, answer): game_correct += 1
                result_df = gsm_to_result_df(result_df, idx, answer, a_first_ans, b_first_ans, ans_a, ans_b, round)
                break
            # majority vote when no converge
            if round == max_round -1:
                try:
                    v = Counter(answers_for_majority)
                    vote_result = v.most_common(1)[0][0]
                except:
                    vote_result = None
                result_df = gsm_to_result_df(result_df, idx, answer, a_first_ans, b_first_ans, vote_result, vote_result, round)
                answers_for_majority.clear()
                # Destructor
                del a_history, b_msgs
                dialog_fp.close()
                break
        pbar.set_description(f"GSM Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")
    print(f"GSM Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")

In [27]:
from ChatLLM import chat_baichuan, chat_qwen, chat_phi, chat_llama, chat_mistral
if __name__ == "__main__":
    if multiprocessing.get_start_method() != 'spawn':
        multiprocessing.set_start_method('spawn', force=True)
    while True:
        try:
            main_gsm(
                model_a=qw_model, 
                model_b=mistral_model, 
                tkz_a=qw_tokenizer, 
                tkz_b=mistral_tokenizer, 
                device_a=qw_device, 
                device_b=mistral_device, 
                chat_model_a=chat_qwen, 
                chat_model_b=chat_mistral
            )
            break
        except torch.cuda.OutOfMemoryError:
            pass
        # except ValueError:
        #     pass

GSM Acc: Game=0.550, qwen1.5-7b=0.497, Mistral-v0.2-7B=0.451: 100%|██████████| 1319/1319 [14:16:33<00:00, 38.96s/it]   

GSM Acc: Game=0.550, qwen1.5-7b=0.497, Mistral-v0.2-7B=0.451


## Arc-easy/challenge

In [12]:
#"""in the form (X) <-> in between the brakets """
"""put your answer ++choice++
A) -> (A)
in the form (A) or (B) or (C) or (D)
"""
from fastparquet import ParquetFile
def parse_arc(df, ix):
    question = df.iloc[ix, 1]
    answer = df.iloc[ix, 2]
    if answer.isdigit():
        answer = chr(ord("A")-1+int(answer)) # map digit to A B C D E
    question = f"Can you answer the following question as accurately as possible? {question}: "
    for i,choice in enumerate(df.iloc[ix, 3]):
        if i>0: question += " "
        question += f"({chr(ord('A')+i)}) {choice}"
    question += " Explain your answer, putting the best one answer choice in the form "
    answer_form = ""
    for i in range(len(df.iloc[ix, 4])):
        if i>0: answer_form += "or "
        answer_form += f"({chr(ord('A')+i)}) "
    question += answer_form + "at the end of your response."
    # print(question, answer)
    # raise RuntimeError
    return question, answer, answer_form

def game_prompt_arc(another_resp, answer_form):
    # if len(agents) == 0:
    #     return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}
    prefix_string = "These are the solutions to the problem from other agents: \n\n One agent solution: ```{}```".format(another_resp)
    prefix_string += "\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. "
    prefix_string += f"Put your answer choice in the form {answer_form}at the end of your response."
    prefix_string += f"Make sure that the last term in the form {answer_form}in your response is your final answer."
    return prefix_string

def parse_choice_ans(input_str):
    pattern = r'\(\s{0,}(\w)\s{0,}\)'
    matches = re.findall(pattern, input_str)
    solution = None
    for match_str in matches[::-1]:
        solution = match_str.upper()
        if solution: break
    return solution

def arc_ans_is_equal(resp, label): return resp == label

def arc_run(model_a, model_b, tkz_a, tkz_b, device_a, device_b, chat_model_a, chat_model_b, difficulty, dataset_name):
    dataset_test_df = ParquetFile(os.path.expanduser(f'~/Corpus/ai2_arc/ARC-{difficulty}/test-00000-of-00001.parquet')).to_pandas()
    # build result df
    result_csv_filename = f"{dataset_name}-{difficulty}_{model_a_name}_{model_b_name}_{timestamp}_{max_round}_Last{save_last_n_round}Round.csv"
    csv_path = os.path.join(result_dir, result_csv_filename)
    if os.path.exists(csv_path):
        result_df = pd.read_csv(csv_path)
    else:
        result_df = pd.DataFrame(columns=['ith','label','a_first_ans','b_first_ans','a_game_ans','b_game_ans','rounds'])
        result_df.to_csv(csv_path, index=False)
    def to_result_df(result_df, i, answer, a_first_ans, b_first_ans, a_game_answer, b_game_answer, g):
        result_df = pd.concat([result_df, pd.DataFrame([{
                'ith':i, 'label':answer,
                'a_first_ans':a_first_ans,'b_first_ans':b_first_ans,
                'a_game_ans':a_game_answer,'b_game_ans':b_game_answer,'rounds':g
            }])],ignore_index=True)
        result_df.to_csv(os.path.join(result_dir, result_csv_filename), index=False)
        return result_df

    restart = True
    for idx in (pbar:=tqdm(range(len(dataset_test_df)))):
        if len(result_df)>0:    # skip processed line by line
            if idx in result_df['ith'].values: continue
            if restart:
                label_so_far = result_df.label.to_list()
                model_a_so_far = result_df.a_first_ans.to_list()
                model_b_so_far = result_df.b_first_ans.to_list()
                game_a_so_far = result_df.a_game_ans.to_list()
                game_b_so_far = result_df.b_game_ans.to_list()
                a_correct = list(map(lambda x: arc_ans_is_equal(x[0],x[1]), zip(label_so_far, model_a_so_far))).count(True)
                b_correct = list(map(lambda x: arc_ans_is_equal(x[0],x[1]), zip(label_so_far, model_b_so_far))).count(True)
                game_correct = list(map(lambda x: arc_ans_is_equal(x[0],x[1]) and arc_ans_is_equal(x[2],x[1]), 
                                        zip(label_so_far, game_a_so_far, game_b_so_far))).count(True)
                restart = False
        else:
            a_correct, b_correct, game_correct, subject_start_idx = 0,0,0,0
        # save dialog
        dialog_dir = os.path.join(result_dir, f"dialog_{timestamp}_LastOnly")
        if not os.path.exists(dialog_dir): os.mkdir(dialog_dir)
        dialog_filename = f"{dataset_name}-{difficulty}_{model_a_name}_{model_b_name}_{timestamp}_{idx}_LastOnly.txt"
        dialog_fp = open(os.path.join(dialog_dir, dialog_filename), 'w+', encoding="utf-8")
        # game rounds
        answers_for_majority = []
        question, answer, answer_form = parse_arc(dataset_test_df,idx)
        for round in range(max_round):
            if round == 0:
                pool = Pool(processes=2)
                result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, question,))
                result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, question,))
                pool.close()
                pool.join()
                a_resp, a_history = result_a.get()
                b_resp, b_msgs = result_b.get()
            else:    # ans_a != ans_b
                if round > save_last_n_round:    # shave history
                    a_history = a_history[:(3 if a_history[0]['role'] == "system" else 2)] + a_history[-2*save_last_n_round:]
                    b_msgs = b_msgs[:(3 if b_msgs[0]['role'] == "system" else 2)] + b_msgs[-2*save_last_n_round:]
                game_a_prompt, game_b_prompt = game_prompt_arc(b_resp, answer_form), game_prompt_arc(a_resp, answer_form)
                pool = Pool(processes=2)
                result_a = pool.apply_async(chat_model_a, (model_a, tkz_a, device_a, game_a_prompt, a_history))
                result_b = pool.apply_async(chat_model_b, (model_b, tkz_b, device_b, game_b_prompt, b_msgs))
                pool.close()
                pool.join()
                a_resp, a_history = result_a.get()
                b_resp, b_msgs = result_b.get()
            # save response
            dialog_fp.write(f"a: {a_resp}\nb: {b_resp}\n\n")
            ans_a, ans_b = parse_choice_ans(a_resp), parse_choice_ans(b_resp)
            # save answer for voting
            answers_for_majority.append(ans_a)
            answers_for_majority.append(ans_b)
            # control game flow
            if round == 0: 
                a_first_ans, b_first_ans = ans_a, ans_b
                if arc_ans_is_equal(ans_a, answer.replace(',',"")): a_correct += 1
                if arc_ans_is_equal(ans_b, answer.replace(',',"")): b_correct += 1
            if arc_ans_is_equal(ans_a, ans_b):
                if arc_ans_is_equal(ans_a, answer): game_correct += 1
                result_df = to_result_df(result_df, idx, answer, a_first_ans, b_first_ans, ans_a, ans_b, round)
                break
            # majority vote when no converge
            if round == max_round -1:
                try:
                    v = Counter(answers_for_majority)
                    vote_result = v.most_common(1)[0][0]
                except:
                    vote_result = None
                result_df = to_result_df(result_df, idx, answer, a_first_ans, b_first_ans, vote_result, vote_result, round)
                answers_for_majority.clear()
                # Destructor
                del a_history, b_msgs
                dialog_fp.close()
                break
        pbar.set_description(f"{dataset_name}-{difficulty} Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")
    print(f"{dataset_name}-{difficulty} Acc: Game={game_correct/(idx+1):.3f}, {model_a_name}={a_correct/(idx+1):.3f}, {model_b_name}={b_correct/(idx+1):.3f}")

In [13]:
from ChatLLM import chat_baichuan, chat_qwen, chat_phi, chat_llama, chat_mistral
if __name__ == "__main__":
    if multiprocessing.get_start_method() != 'spawn':
        multiprocessing.set_start_method('spawn', force=True)
    while True:
        try:
            arc_run(
                model_a=baichuan_model, 
                model_b=mistral_model, 
                tkz_a=baichuan_tokenizer, 
                tkz_b=mistral_tokenizer, 
                device_a=baichuan_device, 
                device_b=mistral_device, 
                chat_model_a=chat_baichuan, 
                chat_model_b=chat_mistral,
                # difficulty="Easy",
                difficulty="Challenge",
                dataset_name="ARC"
            )
            break
        except torch.cuda.OutOfMemoryError:
            pass
        # except ValueError:
        #     pass

ARC-Challenge Acc: Game=0.640, Baichuan2-7B=0.511, Mistral-v0.2-7B=0.608: 100%|██████████| 1172/1172 [11:29:47<00:00, 35.31s/it]  

ARC-Challenge Acc: Game=0.640, Baichuan2-7B=0.511, Mistral-v0.2-7B=0.608


In [17]:
chr(ord("A")+1)

'B'